In [15]:
#!pip3 install email-validate
# coding: utf8

In [21]:
import sys
#from imp import reload
#reload(sys)
print(sys.getdefaultencoding())

utf-8


In [22]:
import pandas as pd
import pymssql as psq
import psycopg2.extras
from email_validate import validate, validate_or_fail
import email_validate.exceptions as eex


Соединяемся с базой стек

In [82]:
mes_conn = psq.connect(server='192.168.9.10', user='LUBCHANSKIY', password='789456', database='Magadan_stack', charset='utf8')

In [88]:
def email_fizlic(s_conn):
    cursor = s_conn.cursor()
    sql_srting = "SELECT лс.Номер лицевой, св.Примечание email FROM stack.Свойства св join stack.[Лицевые счета] лс ON лс.ROW_ID = св.[Счет-Параметры] where [Виды-Параметры]=177"
    #sql_srting = sql_srting.encode().decode('windows-1251')
    #print(sql_srting)
    cursor.execute(sql_srting)
    row = cursor.fetchone()
    data = []
    while row:
        data.append([row[0], row[1].encode('windows-1251').decode()])
        row = cursor.fetchone()      
    return data

In [108]:
def email_ulorg(s_conn):
    cursor = s_conn.cursor()
    cursor.execute("SELECT Название, орг.email FROM stack.[Организации] орг where email is not null and email != ''")
    row = cursor.fetchone()
    data = []
    while row:
        data.append([row[0].encode('ISO-8859-1').decode('windows-1251'), row[1]]) #.encode().decode('ISO-8859-1')
        row = cursor.fetchone()      
    return data

In [109]:
fiz_email_list = email_fizlic(mes_conn)
ul_email_list = email_ulorg(mes_conn)

In [7]:
def valid_email(email):
    try:
        validate_or_fail(
            email_address=email,
            check_format=True,
            check_blacklist=False,
            check_dns=True,
            dns_timeout=10,
            check_smtp=False,
            smtp_debug=False
        )
    except eex.DomainNotFoundError:
        return 'Адрес домена не найден'
    except eex.NoNameserverError:
        return ''
    except eex.DNSTimeoutError:
        return 'Истекло время ожидания при запросе сервера имен.'
    except eex.DNSConfigurationError:
        return 'Сервер имен настроен неправильно.'
    except eex.NoMXError:
        return 'Сервер имен не содержит записей MX для домена.'
    except eex.NoValidMXError:
        return 'Сервер имен перечисляет записи MX для домена, но ни одна из них не является допустимой.'
    except eex.AddressFormatError:
        return 'Формат адреса не соответствует требованиям'
    except eex.DomainBlacklistedError:
        return 'Адрес домена в черном списке!'
    else:
        return ''

In [8]:
email_errors = []
for email_ls in fiz_email_list:
    res = valid_email(email_ls[1])
    if res != '':
        email_errors.append([email_ls[0], email_ls[1], res])
email_df = pd.DataFrame(email_errors)
email_df

,0,1,2
0,111408,muchkaeva@mdnab.su,Адрес домена не найден
1,120238,fred_magadan13@maiI.ru,Сервер имен не содержит записей MX для домена.
2,130375,tam_mazur.@mail.ru,Формат адреса не соответствует требованиям
3,222196,olya-olechka-2012@maiI.ru,Сервер имен не содержит записей MX для домена.
4,230136,ivachepele@gmail.ru,Сервер имен не содержит записей MX для домена.
5,240628,naiIevna2015@maiI.ru,Сервер имен не содержит записей MX для домена.
6,241867,inna.aIzhanova@maiI.ru,Сервер имен не содержит записей MX для домена.
7,320349,bev28ym@maiI.ru,Сервер имен не содержит записей MX для домена.
8,321743,jv.turulina@maik.ru,Сервер имен не содержит записей MX для домена.
9,340739,polyarnya@gmali.com,Сервер имен не содержит записей MX для домена.


In [111]:
ul_email_errors = []
for email_ls in ul_email_list:
    if ';' in email_ls[1]:
        spl_res = email_ls[1].split(';')
        for eml in spl_res:
            res = valid_email(eml)
            if res != '':
                ul_email_errors.append([email_ls[0], eml, res])
    elif  ',' in email_ls[1]:
        spl_res = email_ls[1].split(',')
        for eml in spl_res:
            res = valid_email(eml)
            if res != '':
                ul_email_errors.append([email_ls[0], eml, res])
    else:
        res = valid_email(email_ls[1])
        if res != '':
            ul_email_errors.append([email_ls[0], email_ls[1], res])
            
ulemail_df = pd.DataFrame(ul_email_errors)
ulemail_df

,0,1,2
0,"ООО ""Любимый город""",info@ooouklg.ru,Адрес домена не найден
1,ТЕСТОВЫЙ ПЛАТЕЛЬЩИК ЮЛ №1,test@test.ru,Адрес домена не найден
2,"АО ""КОЛЫМСКАЯ РОССЫПЬ""",buhtmc@kolymros.ru,Адрес домена не найден
3,"АО ""ОАВС""",alexgv777@mail.ru,Формат адреса не соответствует требованиям
4,"АО ""ОАВС""",kozlenko777@yandex.ru,Формат адреса не соответствует требованиям
5,"ГКОУ ""МОЦО № 1""",mogkou19@yandex.ru,Формат адреса не соответствует требованиям
6,"ГСК ""Содружество""",zakup@meser.su,Адрес домена не найден
7,ДАЛЬНЕДРА,dalnedra@rosnedrs.gov.ru,Адрес домена не найден
8,"МАУ ""СК""МЕТАЛЛИСТ""",cport.metallict49@mail.ru,Формат адреса не соответствует требованиям
9,"МАУК г. Магадана ""Кинотеатр""Горняк""/",info@kinomagadan.ru,Формат адреса не соответствует требованиям


In [28]:
def enc_str(str_):
    return str_.decode('utf8').encode('windows-1251')

In [29]:
ulemail_df['enc_1'] = ulemail_df[0].apply(enc_str)
ulemail_df

AttributeError: 'str' object has no attribute 'decode'

In [41]:
# Запихиваем все в Excel физиков
sh_name = 'Ошибки'
writer = pd.ExcelWriter('email_error.xlsx', engine='xlsxwriter')

email_df.to_excel(writer, sheet_name=sh_name, index=False)

workbook = writer.book
worksheet = writer.sheets[sh_name]
writer.save()
writer.close()


c:\Users\Lubchanskiy\AppData\Local\Programs\Python\Python310\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [112]:
# Запихиваем все в Excel юриков
sh_name = 'Ошибки'
writer = pd.ExcelWriter('UL_email_error.xlsx', engine='xlsxwriter')

ulemail_df.to_excel(writer, sheet_name=sh_name, index=False)

workbook = writer.book
worksheet = writer.sheets[sh_name]
writer.save()
writer.close()


c:\Users\Lubchanskiy\AppData\Local\Programs\Python\Python310\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [29]:
print(len(fiz_email_list))

2213


In [27]:
ulemail_df.to_csv('ul_error.csv')